In [16]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist

In [17]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Scrape Mars NASA News Site

In [18]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [19]:
news_titles = []
news_ps = []
for x in range(1):

    html = browser.html
    soup = bs(html, "html.parser")

    quotes = soup.find_all("div", class_="content_title")
    pars = soup.find_all("div", class_="rollover_description")

    for quote in quotes:
        news_titles.append(quote.text)
    for par in pars:
        news_ps.append(par.text)


In [20]:
news_title = news_titles[0]
news_p = news_ps[0]


### JPL Mars Space Images

In [21]:
url_2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_2)
html_2 = browser.html
soup_2 = bs(html_2, "html.parser")

In [22]:
#Get info from page
page_info = soup_2.findAll("article", {"class":["carousel_item"]})
picture = [page.get("style") for page in page_info]
img_url = picture[0].replace("background-image: url('","")
img_url = img_url.replace("');","")
featured_image_url = "https://www.jpl.nasa.gov" + img_url

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18048-1920x1200.jpg


### Mars Weather

In [31]:
url_3 = 'https://twitter.com/marswxreport?lang=en'
data = requests.get(url_3)

In [32]:
#loop and pull all tweets. filter weather related and display latestest item.\n",
tweet_stash = []
html = bs(data.text, 'html.parser')
timeline = html.select('#timeline li.stream-item')
for tweet in timeline:
        tweet_text = tweet.select('p.tweet-text')[0].get_text()
        if "sol" in tweet_text:
            tweet_stash.append(tweet_text)
mars_weather = tweet_stash[0]
mars_weather

'InSight sol 137 (2019-04-16) low -97.0ºC (-142.7ºF) high -15.9ºC (3.4ºF)\nwinds from the SW at 4.3 m/s (9.7 mph) gusting to 12.4 m/s (27.7 mph)\npressure at 7.30 hPapic.twitter.com/jhaMb7q9gy'

### Mars Facts

In [25]:
#reading URL and using pandas to retrieve table
url_4 = 'https://space-facts.com/mars/'
tables = pd.read_html(url_4)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [26]:
#extract tables
mars_df = tables[0]
mars_df.columns = ["Mars Planetary Feature", "Facts"]
mars_df.head()

,Mars Planetary Feature,Facts
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [27]:
#turn extracted table back to HTML and remove blank spaces
html_table = mars_df.to_html()
html_table.replace("\n", "")

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars Planetary Feature</th>      <th>Facts</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egypt

### Mars Hemispheres

In [28]:

url_5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
base_url = 'https://astrogeology.usgs.gov'
browser.visit(url_5)

In [29]:
#parsing and reading through items to pull image items

titles = []
linklist = []
html = browser.html
soup = bs(html, 'html.parser')
t = soup.find_all('h3')
for x in t:
    titles.append(x.text)

for i in titles:
    browser.click_link_by_partial_text(i)
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = bs(html, 'html.parser')
    # Retrieve all elements that contain book information
    u = soup.find('img', class_ = 'wide-image')
    linklist.append(base_url + u["src"])
    browser.click_link_by_partial_text('Back')
   
 

In [30]:
#organizing items as a dictionary

keys  = ["title", "img_url"]
hemisphere_image_urls = [dict(zip(keys,row)) for row in zip(titles,linklist)]

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
